In [ ]:
import osmnx as ox
from shapely.geometry import LineString, mapping
import geopandas as gpd
from ipyleaflet import *
import timeit
import ipywidgets as widgets
from queue import *
from ipywidgets import Layout
from IPython.display import display, Image
from haversine import haversine
from sklearn.neighbors import BallTree
import pickle

In [ ]:
#Callbacks to set markers to nearest node
def set_nearest_node(marker):
    marker.nearest_node = ox.nearest_nodes(graph, marker.location[1], marker.location[0], return_dist=False)
    return

def fix_location(event):
    marker = event['owner']
    set_nearest_node(marker)
    marker.location = graph.nodes[marker.nearest_node]["y"], graph.nodes[marker.nearest_node]["x"]

In [ ]:
#Pathfinder Algorithms

def min_node(dists, openn):
    n, d = -1, float('inf')
    
    for node in openn:
        dist = dists[node]
        if d > dist:
            n, d = node, dist
    
    return n


def heuristic(node, target, dists):

    to = graph.nodes[target]["y"], graph.nodes[target]["x"],
    fr = graph.nodes[node]["y"], graph.nodes[node]["x"]
    return haversine(to, fr)*(50)

def heuristic2(node, target, dists):

    to = graph.nodes[target]["y"], graph.nodes[target]["x"],
    fr = graph.nodes[node]["y"], graph.nodes[node]["x"]
    return haversine(to, fr)*(10000)

def dijkstra(G, source, target, astar = False, unsafe = False):
    
    openn = {source}
    closed = set()
    dists = {source: 0}
    prev = {source: source}
    
    if not unsafe:
        def heuristic(node, target, dists):

            to = graph.nodes[target]["y"], graph.nodes[target]["x"],
            fr = graph.nodes[node]["y"], graph.nodes[node]["x"]
            return haversine(to, fr)*(50)
    else:
        def heuristic(node, target, dists):

            to = graph.nodes[target]["y"], graph.nodes[target]["x"],
            fr = graph.nodes[node]["y"], graph.nodes[node]["x"]
            return haversine(to, fr)*(10000)
    
    if astar:
        f = {source: heuristic(source, target, dists) + dists[source]}
    
    curr = source
    count = 0
    
    while True:
        
        openn.remove(curr)
        closed.add(curr)
        
        circle = Circle(location = (graph.nodes[curr]["y"], graph.nodes[curr]["x"]),
                        radius = 1,
                        color = node_color.value,
                        fill_color = node_color.value)
        m.add_layer(circle)
        layer_list.add(circle)
        
        count += 1
        
        for v in G.neighbors(curr):
            
            if v not in closed:
                openn.add(v)
                
                dist = graph[curr][v][0]['length']

                if v not in dists or dists[v] > float(dists[curr] + dist):
                    dists[v] = float(dists[curr] + dist)
                    prev[v] = curr
                
                if astar:
                    new_f = dists[v] + heuristic(v, target, dists)
                    if v not in f or new_f < f[v]:
                        f[v] = new_f
                
        
        if curr == target:
            break
            
        if astar:
            curr = min_node(f, openn)
        else:
            curr = min_node(dists, openn)
            
        if curr == -1:
            break
    
    return {'distances': dists, 'prev_vertices': prev,
           'count': count}
            
            
def breadth(G, source, target):
    
    q = Queue()
    q.put(source)
    
    visited = {source: True}
    dists = {source: 0}
    prev = {source: source}
    
    count = 0
    curr = source
    while curr != target:
        curr = q.get()

        circle = Circle(location = (graph.nodes[curr]["y"], graph.nodes[curr]["x"]),
                    radius = 1,
                    color = node_color.value,
                    fill_color = node_color.value)
        m.add_layer(circle)
        layer_list.add(circle)

        count += 1

        for v in G.neighbors(curr):
            if v not in visited or not visited[v]:
                q.put(v)
                visited[v] = True
                prev[v] = curr

                dist = graph[curr][v][0]['length']
                dists[v] = float(dists[curr] + dist)
        
    return {'distances': dists, 'prev_vertices': prev,
               'count': count}


def draw_path(G, source, target, algorithm):
    
    start_time = timeit.default_timer()
    
    if source==target:
        alg = {'distances': {source: 0}, 'prev_vertices': {source: source}, 'count': 0}
    if algorithm == "Dijkstra":
        alg = dijkstra(G, source, target)
    elif algorithm == "Fibonacci Heap Dijkstra":
        alg = dijkstra2(G, source, target)
    elif algorithm == "A* Search":
        alg = dijkstra(G, source, target, True)
    elif algorithm == "Breadth-first Search":
        alg = breadth(G, source, target)
    elif algorithm == "Unsafe A* Search":
        alg = dijkstra(G, source, target, True, True)
        
    prev_vertex = alg['prev_vertices']
    
    curr = target
    path = [curr]
    while curr != source:
        curr = prev_vertex[curr]
        path.append(curr)
        
    final_path = path[::-1]
    print(final_path)
    final_distance = alg['distances'][target]
        
    elapsed = (timeit.default_timer() - start_time) * 1000
    print('>> elapsed time', elapsed, 'ms')
    print('>> count: ', alg['count'])
    print('>> distance: ', alg['distances'][target])
    
    #if len(path_layer_list) == 1:
     #   m.remove_layer(path_layer_list[0])
      #  path_layer_list.pop()
    
    shortest_path_points = nodes.loc[final_path]
    path = gpd.GeoDataFrame([LineString(shortest_path_points.geometry.values)], columns=['geometry'])
    path_layer = GeoData(geo_dataframe=path, style={'color': path_color.value, 'weight': 2})
    m.add_layer(path_layer)
    layer_list.add(path_layer)
    
    return elapsed, alg['count'], alg['distances'][target]
                
    
    

In [ ]:
graph = nodes = edges = center = m = from_marker = to_marker = None
loading = True
layer_list = set()
Toronto = pickle.load(open("Toronto.pickle", "rb"))
New_York = pickle.load(open("New_York.pickle", "rb"))
Mississauga = pickle.load(open("Mississauga.pickle", "rb"))
Paris = pickle.load(open("Paris.pickle", "rb"))
preloaded = {"toronto": Toronto, "paris": Paris, "new york": New_York, "mississauga": Mississauga}

mapLayout=Layout(width='1280px', height='700px')
from_marker_style = AwesomeIcon(
    name='circle',
    icon_color='white',
    marker_color='blue',
    spin=False
)

to_marker_style = AwesomeIcon(
    name='circle',
    icon_color='white',
    marker_color='red',
    spin=False
)

color_layout = widgets.Layout(width='240px')

path_color = widgets.ColorPicker(
            concise=False,
            description='Path Colour:',
            value='black',
            disabled=False,
            layout = color_layout
            )

path_control = WidgetControl(widget=path_color, position='bottomright')

node_color = widgets.ColorPicker(
            concise=False,
            description='Node Colour:',
            value='red',
            disabled=False,
            layout = color_layout
            )

node_control = WidgetControl(widget=node_color, position='bottomright')

time_label = widgets.Label(value="")
count_label = widgets.Label(value="")
distance_label = widgets.Label(value="")

def loadmap(wdgt):
    global graph
    global nodes
    global edges
    global center
    global m
    global from_marker
    global to_marker
        
    loading = widgets.IntProgress(
    value=10,
    min=0,
    max=10,
    description='Loading:',
    bar_style='', # 'success', 'info', 'warning', 'danger' or ''
    style={'bar_color': 'maroon'},
    orientation='horizontal'
    )
    
    box.children = [box.children[0], loading]
    

    try:
        
        #Loading graph
        if (wdgt.value.lower() in preloaded):
            graph = preloaded[wdgt.value.lower()]
        else:  
            graph = ox.graph_from_place(wdgt.value, network_type = 'drive')
        nodes, edges = ox.graph_to_gdfs(graph)
    
        #Setting up map
        center = graph.nodes[nodes.index[0]]["y"], graph.nodes[nodes.index[0]]["x"]
        m = Map(center = center, 
        basemap = basemaps.CartoDB.Positron,
        zoom = 20,
        scroll_wheel_zoom = True, 
        layout = mapLayout)
    
        #Setting up markers
        from_marker = Marker(location=center, icon=from_marker_style)
        to_marker = Marker(location=center, icon=to_marker_style)
        from_marker.observe(lambda event: fix_location(event), 'location')
        to_marker.observe(lambda event: fix_location(event), 'location')
        m.add_layer(from_marker)
        m.add_layer(to_marker)
        set_nearest_node(from_marker)
        set_nearest_node(to_marker)
    
        #Setting up map widgets and interactivity
               
        dropdown = widgets.Dropdown(
            options=["Dijkstra", 'A* Search', 'Unsafe A* Search', 'Breadth-first Search'],
            value="Dijkstra",
            description='Algorithm:',
            tooltip='Select algorithm to visualize',
            disabled=False,
            )

        dropdown_control = WidgetControl(widget=dropdown, position='topright')


        layout = widgets.Layout(width='100px', height='30px')

        button = widgets.Button(
        description='Visualize',
        disabled=False,
        button_style='', # 'success', 'info', 'warning', 'danger' or ''
        tooltip='Click to visualize selected pathfinding algorithm',
        icon='play', # (FontAwesome names without the `fa-` prefix)
        layout = layout
        )

        def button_action(arg):
            global output_label
            x = draw_path(graph, from_marker.nearest_node, to_marker.nearest_node, dropdown.value)
            time_label.value = "elapsed time: " + str(x[0]) + ' ms'
            count_label.value = "nodes traversed: " + str(x[1])
            distance_label.value = "total path distance: " + str(x[2]) + " m"


        button.on_click(button_action)
        button_control = WidgetControl(widget=button, position='topright')

        clear = widgets.Button(
        description='Clear Map',
        disabled=False,
        button_style='', # 'success', 'info', 'warning', 'danger' or ''
        tooltip='Clear nodes and path from map',
        icon='trash', # (FontAwesome names without the `fa-` prefix)
        layout = layout
        )

        def clear_action(arg):
            while len(layer_list):
                m.remove_layer(layer_list.pop())


        clear.on_click(clear_action)
        clear_control = WidgetControl(widget=clear, position='topright')

        m.add_control(dropdown_control)
        m.add_control(button_control)
        m.add_control(clear_control)
        m.add_control(path_control)
        m.add_control(node_control)

        box.children = [box.children[0], widgets.Label(value="Total Nodes: " + str(len(nodes))),
                        widgets.Label(value="Total Edges: " + str(len(edges))), 
                        m, time_label, count_label, distance_label]

    except:
        box.children = [box.children[0], widgets.Label(value="Couldn't find location try again (e.g. Budapest, Pest megye, Hungary)")]



<h1><center>Pathfinder Visualization on Real Map</center></h1>

In [ ]:


regionInput = widgets.Text(
value='Toronto',
placeholder='Type a Location',
description='Location:',
disabled=False   
)   

regionInput.on_submit(loadmap)

box_layout = widgets.Layout(display='flex',
                flex_flow='column',
                width='100%')

box = widgets.HBox([regionInput], layout=box_layout)
display(box)
#"C:\Users\taha4\Pathfinder Visualization Applied to Real Life.ipynb"

<h3>1. Type in a city or an address (e.g. Toronto or Budapest, Pest megye, Hungary) to load that locations nodes and edges into the map.</h3>

<h3>Note: Larger cities can take a long time to load while smaller ones load within a few seconds. And one can only navigate in boundaries of the entered location.</h3>

<h3>2. Once the map is loaded, a node will represent each intersection, while a road between two intersections is an edge.</h3>

<h3>3. You can select a pathfinding algorithm and hit visualize to draw the path and see the nodes traversed.</h3>

<h3>4. To compare different pathfinding algorithms, you can change the node and path color.</h3>

<h3>5. Hit the clear map button to delete previously drawn visualizations.</h3>